# Classification with VGG16 : Cats vs Dogs

---
```
Step 0. Library Import
Step 1. Load Dataset
Step 2. Data Preprocessing
Step 3. Modeling
Step 4. Model Compile
Step 5. Model Checkpoint
Step 6. Model Fit
Step 7. Model Evaluate & Save
Step 8. Model Reload
```

## Step 0. Library Import

In [3]:
import tensorflow as tf

import tensorflow_datasets as tfds

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

## Step 1. Load Dataset


In [4]:
name = 'cats_vs_dogs'

train_set = tfds.load(name=name, split='train[:80%]')
valid_set = tfds.load(name=name, split='train[80%:]')

## Step 2. Data Preprocessing

In [5]:
def preprocess(data):
    x = data['image'] / 255
    x = tf.image.resize(x, size=(224,224))

    y = data['label']

    return x, y

In [6]:
batch_size=128
train_data = train_set.map(preprocess).batch(batch_size)
valid_data = valid_set.map(preprocess).batch(batch_size)

## Step 3. Modeling

In [7]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
vgg.trainable=False

In [8]:
model = Sequential([
    vgg,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 27,561,282
Trainable params: 12,846,594
Non-trainable params: 14,714,688
_________________________________________________________________


## Step 4. Model Compile

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

## Step 5. Model Checkpoint

In [11]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

## Step 6. Model Fit

In [12]:
epochs=12
model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    callbacks=[cp]
)

Epoch 1/12
146/146 [==============================] - 60s 329ms/step - loss: 1.4721 - acc: 0.7675 - val_loss: 0.1830 - val_acc: 0.9243

Epoch 00001: val_loss improved from inf to 0.18304, saving model to my_checkpoint.ckpt
Epoch 2/12
146/146 [==============================] - 44s 301ms/step - loss: 0.1900 - acc: 0.9193 - val_loss: 0.1738 - val_acc: 0.9265

Epoch 00002: val_loss improved from 0.18304 to 0.17381, saving model to my_checkpoint.ckpt
Epoch 3/12
146/146 [==============================] - 42s 283ms/step - loss: 0.1601 - acc: 0.9352 - val_loss: 0.1579 - val_acc: 0.9357

Epoch 00003: val_loss improved from 0.17381 to 0.15794, saving model to my_checkpoint.ckpt
Epoch 4/12
146/146 [==============================] - 45s 306ms/step - loss: 0.1468 - acc: 0.9378 - val_loss: 0.1692 - val_acc: 0.9295

Epoch 00004: val_loss did not improve from 0.15794
Epoch 5/12
146/146 [==============================] - 45s 306ms/step - loss: 0.1452 - acc: 0.9401 - val_loss: 0.1867 - val_acc: 0.9235



## Step 7. Model Evaluate & Save

In [13]:
model.load_weights(filepath)
model.evaluate(valid_data)

37/37 [==============================] - 8s 216ms/step - loss: 0.1579 - acc: 0.9357


[0.15793825685977936, 0.9357265830039978]

In [14]:
model.save('CvD_vgg16_1579.h5')

## Step 8. Model Reload


In [15]:
mymodel = tf.keras.models.load_model('CvD_vgg16_1579.h5')
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 27,561,282
Trainable params: 12,846,594
Non-trainable params: 14,714,688
_________________________________________________________________
